In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/standard/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch
import torchvision
from torchvision import datasets, models, transforms
import os
from torch.autograd import Variable
import matplotlib.pyplot as plt
from torch.utils.data import SubsetRandomSampler
import time
import numpy as np
%matplotlib inline

In [ ]:
data_transform = transforms.Compose([transforms.Scale([224,224]),
                                        transforms.Resize([224,224]),
                                        transforms.ToTensor(),
                                        transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5])])

image_datasets = datasets.ImageFolder(root ="/kaggle/input/traindf/train",transform = data_transform)
print(image_datasets)
N=len(image_datasets)
print(N)
batch_size=16

In [ ]:
validation_split = .01
shuffle_dataset = True
random_seed= 42

# Creating data indices for training and validation splits:
dataset_size = len(image_datasets)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)

In [ ]:
train_indices_1,val_indices_1 = indices[split:],indices[:split]
val_indices_2 = indices[split:2*split]
train_indices_2=[]
train_indices_3=[]
train_indices_4=[]
train_indices_5=[]
for i in indices:
    if i not in val_indices_2:
        train_indices_2.append(i)
val_indices_3=indices[2*split:3*split]
for i in indices:
    if i not in val_indices_3:
        train_indices_3.append(i)
val_indices_4=indices[3*split:4*split]
for i in indices:
    if i not in val_indices_4:
        train_indices_4.append(i)
val_indices_5=indices[4*split:]
for i in indices:
    if i not in val_indices_5:
        train_indices_5.append(i)

In [ ]:
# Creating PT data samplers and loaders:
train_data = SubsetRandomSampler(train_indices_2)
valid_data = SubsetRandomSampler(val_indices_2)

train_loader = torch.utils.data.DataLoader(image_datasets, batch_size=batch_size, 
                                           sampler=train_data)
val_loader = torch.utils.data.DataLoader(image_datasets, batch_size=batch_size,
                                                sampler=valid_data)

In [ ]:
X_example, y_example = next(iter(train_loader))
example_classes = image_datasets.classes
index_classes = image_datasets.class_to_idx
print(example_classes)
print(index_classes)

In [ ]:
classes_index = {value:key for key,value in index_classes.items()}

In [ ]:
# 根据value得到key
list (index_classes.keys()) [list (index_classes.values()).index (172)]

In [ ]:
img=torchvision.utils.make_grid(X_example)
img=img.numpy().transpose([1,2,0])
print([example_classes[i] for i in y_example])
plt.imshow(img)
plt.show()

In [ ]:
model = models.densenet121(pretrained=True)
Use_gpu = torch.cuda.is_available()
device = torch.device("cuda:0" if torch.cuda.is_available else "cpu")
print(model)

In [ ]:
#!pip install efficientnet_pytorch
#from efficientnet_pytorch import EfficientNet
#model=EfficientNet.from_pretrained('efficientnet-b5')
#print(model)

In [ ]:
for parma in model.parameters():
    parma.requires_grad = True
    model.classifier = torch.nn.Sequential(torch.nn.Linear(1024, 512),
                                           torch.nn.Dropout(p=0.5),
                                           torch.nn.Linear(512, 473))
if Use_gpu:
    model = model.cuda()
model.to(device)

In [ ]:
dataloader={
    "train":train_loader,
    "valid":val_loader
}
img_datasets={
    "train":train_data,
    "valid":valid_data
}

In [ ]:
cost = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())
loss_f = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.000001)
epoch_n = 2

In [ ]:
time_open = time.time()
for epoch in range(epoch_n):
    print("Epoch {}/{}".format(epoch, epoch_n - 1))
    print("-"*10)
    
    for phase in ["train", "valid"]:
        if phase == "train":
            print("Training...")
            model.train(True)
        else:
            print("Validing...")
            model.train(False)
            
        running_loss = 0.0
        running_corrects = 0
    
        for batch, data in enumerate(dataloader[phase], 1):
            X, y = data
            if Use_gpu:
                X, y = Variable(X.cuda()), Variable(y.cuda())
            else:
                X, y = Variable(X), Variable(y)
            X = X.to(device)
            y = y.to(device)
        
            y_pred = model(X)
            _, pred = torch.max(y_pred.data, 1)
            optimizer.zero_grad()
            loss = loss_f(y_pred, y)
        
            if phase == "train":
                loss.backward()
                optimizer.step()
            
            running_loss += loss.data
            running_corrects += torch.sum(pred == y.data)
               
            if batch%500 == 0 and phase =="train":
                print("Batch {}, Train Loss:{:.4f}, Train ACC:{:.4f}%".format(batch, running_loss/batch, 100*running_corrects/(16*batch)))

        epoch_loss = running_loss*16/len(img_datasets[phase])
        epoch_acc = 100*running_corrects/len(img_datasets[phase])
        print("{} Loss:{:.4f} Acc:{:.4f}%".format(phase, epoch_loss,epoch_acc))
            
time_end = time.time() - time_open
print(time_end)

In [ ]:
torch.save(model,"efficientnet1.pkl")

In [ ]:
import torch
model = torch.load('/kaggle/input/vggmodel/model.pkl')

In [ ]:
test_datasets = datasets.ImageFolder(root ="/kaggle/input/testdf/test_integrate",
                                         transform = data_transform)
print(len(test_datasets))
n=len(test_datasets)
test_loader=torch.utils.data.DataLoader(test_datasets,batch_size=15,shuffle=False)

In [ ]:
test_clasees = test_datasets.classes
test_classes = test_datasets.class_to_idx
print(test_clasees)
print(test_classes)

In [ ]:
Use_gpu = torch.cuda.is_available()
device = torch.device("cuda:0" if torch.cuda.is_available else "cpu")

In [ ]:
prediction=[]
for i, data in enumerate(test_loader):
    X, y = data
    X, y = Variable(X.cuda()), Variable(y.cuda())
    X = X.to(device)
    y = y.to(device)
    y_pred = model(X)
    _, pred = torch.max(y_pred.data, 1)
    predict=pred.data.cpu().numpy()
    #print(predict)
    prediction.append(predict)

In [ ]:
prediction=[]
correct=0
for i, data in enumerate(test_loader):
    X, y = data
    X, y = Variable(X.cuda()), Variable(y.cuda())
    X = X.to(device)
    y = y.to(device)
    y_pred = model(X)
    maxk=max((1,3))
    y_resize = y.view(-1,1)
    _,pred=y_pred.topk(maxk,1,True,True)
    correct+=torch.eq(pred,y_resize).sum().float().item()
    predict=pred.data.cpu().numpy()
    print(predict)
    prediction.append(predict)

In [ ]:
correct/n

In [ ]:
prediction

In [ ]:
def evaluteTop5(model,test_loader):
    model.eval()
    correct=0
    for x,y in test_loader:
        x,y=x.to(device),y.to(device)
        with torch.no_grad():
            logits=model(x)
            maxk=max((1,5))
            y_resize=y.view(-1,1)
            _,pred=logits.topk(maxk,1,True,True)
            correct+=torch.eq(pred,y_resize).sum().float().item()
    return correct

In [ ]:
T = evaluteTop5(model,test_loader)

In [ ]:
print(T)

In [ ]:
# 根据value得到key
#list (index_classes.keys()) [list (index_classes.values()).index (123)]

In [ ]:
img_test=torchvision.utils.make_grid(X)
img_test=img_test.numpy().transpose([1,2,0])
#print([test_classes[i] for i in y])
plt.figure(figsize=(12,80))
plt.imshow(img_test)
plt.show()

In [ ]:
pred=[]
for i in range(len(prediction)):
    M = prediction[i]
    for k in M:
        pred.append(k)


In [ ]:
pred_class=[]
for i in pred:
    d=classes_index[i]
    pred_class.append(d)

In [ ]:
p = np.arange(0,474)
True_label=[]
for i in range(473):
    s=classes_index[i]
    True_label.append(s)
len(True_label)

In [ ]:
import pandas as pd
X={"True_label":True_label,"Pred_label":pred_class}
X=pd.DataFrame(X)
X.to_csv("standard.csv")

In [ ]:
import pandas as pd
true=[]
for i in range(len(prediction)):
    sss=prediction[i]
    true.append(sss)

#true=np.array(true)
#X=pd.DataFrame(true,columns=["Label"])
#X.to_csv("X.csv")

In [ ]:
p = np.arange(0,474)
true=np.array(true)
t=0
F={}
for i in true:
    F[i]=i

Q={}
for i in p:
    Q[i]=i

In [ ]:
p = np.arange(0,474)
A=[]
B=[]
for i in p:
    A.append(i)
for j in true:
    B.append(j)

In [ ]:
for i in range(474):
    print(A[i]==B[i])